### NFL play to play data procesing and exploratory analysis
* The NFL play by play data from the year 2018 to 2021 can be directly downloaded from [NFL Savant website](http://nflsavant.com/about.php)
* It contains every play information of all games of the 32 NFL teams from 2018 to 2021 season
* Data from all four seasons will be combined, and further processing for exploratory data anlaysis

### Import modules

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

### Raw Data preprocessing Step
* Combine data from year 2018 to 2021 into one dataset, look into dtype and missing values of each column,
remove columns that provide zero inforamtion and re-organize rest columns


#### Step 1 Read data

In [31]:
#read the NFL data (from 2018 - 2021)

data2021 = pd.read_csv("NFL_play_by_play_data/pbp-2021.csv")
data2020 = pd.read_csv("NFL_play_by_play_data/pbp-2020.csv")
data2019 = pd.read_csv("NFL_play_by_play_data/pbp-2019.csv")
data2018 = pd.read_csv("NFL_play_by_play_data/pbp-2018.csv")



#check the shape of the data
print(data2021.shape)
print(data2020.shape)
print(data2019.shape)
print(data2018.shape)

(42795, 45)
(46189, 45)
(42186, 45)
(45016, 45)


In [32]:
#check if any columns are different among data
print(sum(data2021.columns != data2020.columns))
print(sum(data2020.columns != data2019.columns))
print(sum(data2019.columns != data2018.columns))
print("All columns are matching!")

0
0
0
All columns are matching!


#### Step 2 Combine datasets

In [33]:
#concat all four subsets by columns

nfl_data = pd.concat([data2021, data2020, data2019, data2018], axis=0)

print(f"The combined data shape is {nfl_data.shape}")

nfl_data.head().T

The combined data shape is (176186, 45)


,0,1,2,3,4
GameId,2021092612,2021092612,2021092612,2021092612,2021101008
GameDate,2021-09-26,2021-09-26,2021-09-26,2021-09-26,2021-10-10
Quarter,1,1,1,1,1
Minute,9,8,7,7,9
Second,10,32,52,13,50
OffenseTeam,MIN,MIN,MIN,MIN,WAS
DefenseTeam,SEA,SEA,SEA,SEA,NO
Down,1,1,2,1,2
ToGo,10,10,2,8,2
YardLine,57,74,82,92,51


#### Step 3 Check column infos and investigate on missing values

In [34]:
#Call the info method on NFL data to check the summary of the data
nfl_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176186 entries, 0 to 45015
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   GameId                          176186 non-null  int64  
 1   GameDate                        176186 non-null  object 
 2   Quarter                         176186 non-null  int64  
 3   Minute                          176186 non-null  int64  
 4   Second                          176186 non-null  int64  
 5   OffenseTeam                     161997 non-null  object 
 6   DefenseTeam                     168543 non-null  object 
 7   Down                            176186 non-null  int64  
 8   ToGo                            176186 non-null  int64  
 9   YardLine                        176186 non-null  int64  
 10  Unnamed: 10                     0 non-null       float64
 11  SeriesFirstDown                 176186 non-null  int64  
 12  Unnamed: 12      

In [35]:
#select object features and numeric features
object_features = nfl_data.select_dtypes(include='object').columns
numeric_features = nfl_data.select_dtypes(exclude='object').columns


In [36]:
#Check of number of missing data per column for object and numeric features


missing = pd.concat([nfl_data.isnull().sum(), 100 * nfl_data.isnull().mean()], axis=1)
missing.columns=['count',  '%missing']
pd.set_option('display.max_rows', missing.shape[0]+1)
sorted_missing = missing.sort_values(by='count', ascending=False)


print(f"Missing data in object features:\n{sorted_missing.loc[list(object_features)].sort_values('%missing')}")

print(f"Missing data in numeric features:\n{sorted_missing.loc[list(numeric_features)].sort_values('%missing')}")

Missing data in object features:
                    count   %missing
GameDate                0   0.000000
Description             0   0.000000
YardLineDirection       0   0.000000
Formation            3502   1.987672
PlayType             7098   4.028697
DefenseTeam          7643   4.338029
OffenseTeam         14189   8.053421
PassType           103823  58.928065
RushDirection      128460  72.911582
PenaltyType        162023  91.961336
PenaltyTeam        162025  91.962471
Missing data in numeric features:
                                 count  %missing
GameId                               0       0.0
IsPenaltyAccepted                    0       0.0
YardLineFixed                        0       0.0
IsTwoPointConversionSuccessful       0       0.0
IsTwoPointConversion                 0       0.0
IsPenalty                            0       0.0
IsFumble                             0       0.0
IsInterception                       0       0.0
IsMeasurement                        0       0.0

In [37]:
'''
Removing all unnamed columns (just separation line in the csv table)
Remove Challenger and penalties from the raw dataset

'''

filtered_data = nfl_data.drop(['Challenger', 'Unnamed: 12', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 10'\
                              ,'PenaltyType', 'PenaltyTeam'], axis=1)




In [38]:
'''
Explore the reasons of missing values in Formation and PlayType
'''

#Most missing Formation are not related with game play itself

print(f"Missing Formation are not related with game play itself:\n\
{filtered_data[filtered_data.Formation.isnull()].value_counts('Description')}")




Missing Formation are not related with game play itself:
Description
END QUARTER 1                                                                               987
END QUARTER 3                                                                               986
END QUARTER 2                                                                               981
GAME                                                                                        496
END QUARTER 4                                                                                50
END QUARTER 2 5TH CONSECUTIVE 1ST-HALF SHUTOUT (INCLUDING PLAYOFFS)                           1
END QUARTER 4 TENN CALLED TAILS. COIN TOSS WAS HEADS. BAL WINS TOSS AND WILL RECEIVE KO.      1
dtype: int64


In [39]:
'''
Explore the reasons of missing values in PlayType
'''

##Missing PlayType are not related with game play itself

print(f"Missing PlayType are not related with game play itself:\n\
{filtered_data[filtered_data.PlayType.isnull()].value_counts('Description')}")

Missing PlayType are not related with game play itself:
Description
TWO-MINUTE WARNING                                                                                                      1973
END QUARTER 1                                                                                                            987
END QUARTER 3                                                                                                            986
END QUARTER 2                                                                                                            981
END GAME                                                                                                                 980
                                                                                                                        ... 
(:08) (SHOTGUN) DIRECT SNAP TO 30-D.JOHNSON.  30-D.JOHNSON LEFT END TO PIT 39 FOR 1 YARD (97-C.HEYWARD).                   1
(:06) (SHOTGUN) DIRECT SNAP TO 41-A.KAMARA.  41-A.KAMARA 

In [40]:
'''
Explore the reasons of missing values in DefenseTeam
'''

##Missing DefenseTeamare not related with game play itself

print(f"Missing PlayType are not related with game play itself:\n\
{filtered_data[filtered_data.DefenseTeam.isnull()].value_counts('Description')}")

Missing PlayType are not related with game play itself:
Description
TWO-MINUTE WARNING             1005
GAME                            496
END QUARTER 1                   496
END GAME                        496
END QUARTER 3                   496
                               ... 
TIMEOUT #1 BY PHI AT 07:25.       1
TIMEOUT #1 BY PHI AT 07:45.       1
TIMEOUT #1 BY PHI AT 07:47.       1
TIMEOUT #1 BY PHI AT 07:49.       1
TIMEOUT #2 BY DAL AT 00:24.       1
Length: 3805, dtype: int64


In [41]:
'''
Explore the reasons of missing values in OffenseTeam
'''

##Missing DefenseTeamare not related with game play itself

print(f"Missing PlayType are not related with game play itself:\n\
{filtered_data[filtered_data.OffenseTeam.isnull()].value_counts('Description')}")

Missing PlayType are not related with game play itself:
Description
TWO-MINUTE WARNING             1973
END QUARTER 1                   987
END QUARTER 3                   986
END QUARTER 2                   981
END GAME                        980
                               ... 
TIMEOUT #1 BY OAK AT 08:32.       1
TIMEOUT #1 BY OAK AT 08:17.       1
TIMEOUT #1 BY OAK AT 08:16.       1
TIMEOUT #1 BY OAK AT 08:11.       1
TIMEOUT #2 BY CHI AT 05:52.       1
Length: 6772, dtype: int64


In [42]:
#remove data where Offense or Defense are NaN, as they are not plays
filtered_data = filtered_data.dropna(axis=0, how='any', subset=['OffenseTeam', 'DefenseTeam'])

In [43]:
#removing all irrelavent rows from fitlered data

filtered_data_v2 = filtered_data[~filtered_data[['Formation', 'PlayType', 'OffenseTeam', 'DefenseTeam']].isna().any(axis=1)]
filtered_data_v2.isna().sum().T

GameId                                 0
GameDate                               0
Quarter                                0
Minute                                 0
Second                                 0
OffenseTeam                            0
DefenseTeam                            0
Down                                   0
ToGo                                   0
YardLine                               0
SeriesFirstDown                        0
NextScore                              0
Description                            0
TeamWin                                0
SeasonYear                             0
Yards                                  0
Formation                              0
PlayType                               0
IsRush                                 0
IsPass                                 0
IsIncomplete                           0
IsTouchdown                            0
PassType                           89455
IsSack                                 0
IsChallenge     

In [44]:
'''
Checking the missing values for PassType and RushDirection
'''
#Remove entries where both PassType and RushDirection are null as they are TouchBacks

print("Remove entries where both PassType and RushDirection are null as they are TouchBacks\n")

print(filtered_data_v2[np.logical_and(filtered_data_v2.RushDirection.isnull(), filtered_data_v2.PassType.isnull())]\
.value_counts('Description'))





filtered_data_v3 = filtered_data_v2[~np.logical_and(filtered_data_v2.RushDirection.isnull(), filtered_data_v2.PassType.isnull())]


print(f"\n\nThe final data shape is {filtered_data_v3.shape}")

Remove entries where both PassType and RushDirection are null as they are TouchBacks

Description
7-H.BUTKER KICKS 65 YARDS FROM KC 35 TO END ZONE, TOUCHBACK.                                                                                                                                                                                                                250
8-B.PINION KICKS 65 YARDS FROM TB 35 TO END ZONE, TOUCHBACK.                                                                                                                                                                                                                230
9-J.TUCKER KICKS 65 YARDS FROM BAL 35 TO END ZONE, TOUCHBACK.                                                                                                                                                                                                               205
7-J.SANDERS KICKS 65 YARDS FROM MIA 35 TO END ZONE, TOUCHBACK.                        

In [45]:
#check what rows corresponding to IsNoPlay=1
filtered_data_v3.loc[filtered_data_v3.IsNoPlay==1,].Description[74]

'(12:44) (NO HUDDLE, SHOTGUN) 1-T.TAGOVAILOA PASS SHORT LEFT TO 81-D.SMYTHE TO CAR 23 FOR 3 YARDS (21-J.CHINN). PENALTY ON MIA-74-L.EICHENBERG, OFFENSIVE HOLDING, 10 YARDS, ENFORCED AT CAR 26 - NO PLAY.'

In [46]:
#Remove rows with IsNoPlay==1 as those are plays that were invidated due to offense or defense violations

filtered_data_v3 = filtered_data_v3.loc[filtered_data_v3.IsNoPlay==0,:]


In [47]:
#Check if all NFL teams are included (Raiders, Chargers and Rams Changed its name in the year 2020!)

print(f"Total Team listed in Offense is {len(set(filtered_data_v3.OffenseTeam))}")
print(f"Total Team listed in Defense is {len(set(filtered_data_v3.DefenseTeam))}")
print(f"Same Teams listed in Offense and in Defense {set(filtered_data_v3.DefenseTeam) == set(filtered_data_v3.OffenseTeam)}")
#

Total Team listed in Offense is 32
Total Team listed in Defense is 32
Same Teams listed in Offense and in Defense True


#### Step 4 Double-checking missing values after filtering

In [21]:
filtered_data_v3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114827 entries, 0 to 44993
Data columns (total 38 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   GameId                          114827 non-null  int64 
 1   GameDate                        114827 non-null  object
 2   Quarter                         114827 non-null  int64 
 3   Minute                          114827 non-null  int64 
 4   Second                          114827 non-null  int64 
 5   OffenseTeam                     114827 non-null  object
 6   DefenseTeam                     114827 non-null  object
 7   Down                            114827 non-null  int64 
 8   ToGo                            114827 non-null  int64 
 9   YardLine                        114827 non-null  int64 
 10  SeriesFirstDown                 114827 non-null  int64 
 11  NextScore                       114827 non-null  int64 
 12  Description                    

### Automated exploratory analysis using pandas profile with filtered data

In [96]:
df = filtered_data_v3
    

#I am not seen the differences with explorative=True???
profile_report = df.profile_report(explorative=True, html={'style': {'full_width': True}})
profile_report

#How to display the widget instead???
profile_report.to_widgets()

Summarize dataset:   0%|          | 0/53 [00:00<?, ?it/s]

/root/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
/root/anaconda3/lib/python3.8/site-packages/pandas_profiling/model/correlations.py:101: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [97]:
profile_report.to_file("NFL_data_wrangling.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Remove unneccesary columns, convert categorical features to one hot vector

In [48]:
#drop NextScore, TeamWin, IsMeasurement, IsNoPlay, IsSack, IsTwoPointConversion, \
#IsTwoPointConversionSuccessful columns as they are constant
final_data = filtered_data_v3.drop(axis=1, \
                                   columns=["NextScore", "TeamWin", "IsMeasurement", \
                                            "IsNoPlay", "IsSack", "IsTwoPointConversion", \
                                            "IsTwoPointConversionSuccessful"])

In [49]:
"""
remove columns IsIncomplete IsTouchdown IsChallenge IsChallengeReversed IsInterception IsFumble IsPenalty 
YardLineFixed YardLineDirection IsPenaltyAccepted PenaltyYards
"""

final_data = final_data.drop(axis=1, columns=["IsIncomplete", "IsTouchdown", "IsChallenge", 
                                              "IsChallengeReversed", "IsInterception", "IsFumble", 
                                              "IsPenalty", "YardLineFixed", "YardLineDirection", 
                                              "IsPenaltyAccepted", "PenaltyYards"])

print(final_data.head())

       GameId    GameDate  Quarter  Minute  Second OffenseTeam DefenseTeam  \
0  2021092612  2021-09-26        1       9      10         MIN         SEA   
1  2021092612  2021-09-26        1       8      32         MIN         SEA   
2  2021092612  2021-09-26        1       7      52         MIN         SEA   
3  2021092612  2021-09-26        1       7      13         MIN         SEA   
4  2021101008  2021-10-10        1       9      50         WAS          NO   

   Down  ToGo  YardLine  SeriesFirstDown  \
0     1    10        57                1   
1     1    10        74                0   
2     2     2        82                1   
3     1     8        92                0   
4     2     2        51                1   

                                         Description  SeasonYear  Yards  \
0  (9:10) 8-K.COUSINS PASS SHORT RIGHT TO 83-T.CO...        2021     17   
1  (8:32) 8-K.COUSINS PASS SHORT LEFT TO 18-J.JEF...        2021      8   
2  (7:52) (SHOTGUN) 8-K.COUSINS PASS SHOR

### Build a column transformer to convert all categorical columns to one-hot encoders
* including Offense, Defense Team, Formation, PlayType, PassType and RushDiretion

In [50]:
#self define onehot convert that returns a dataframe with proper column names
class onehot_with_cname(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return a pandas data frame from X
        oe = OneHotEncoder(sparse=False)

        return_df = oe.fit_transform(X)
        
        return_df = pd.DataFrame(return_df, columns=oe.get_feature_names_out(X.columns))
        
        #drop columns that has nan
        return_df = return_df.loc[:,['_nan' not in x for x in return_df.columns]]
        
        return return_df



cat_columns = ["OffenseTeam", "DefenseTeam",  "Formation", "PlayType", "PassType", "RushDirection"]

"""
How to return a pd dataframe with ColumnTransformer???

one_hot_converter = ColumnTransformer([('one_hot', onehot_with_cname(), cat_columns)], \
                                      remainder='passthrough', verbose_feature_names_out=True)
"""

"\nHow to return a pd dataframe with ColumnTransformer???\n\none_hot_converter = ColumnTransformer([('one_hot', onehot_with_cname(), cat_columns)],                                       remainder='passthrough', verbose_feature_names_out=True)\n"

In [51]:
oe = onehot_with_cname()

one_hot_final = oe.fit_transform(final_data[cat_columns])

In [52]:
print(one_hot_final.shape)
one_hot_final.head()


(114827, 95)


,OffenseTeam_ARI,OffenseTeam_ATL,OffenseTeam_BAL,OffenseTeam_BUF,OffenseTeam_CAR,OffenseTeam_CHI,OffenseTeam_CIN,OffenseTeam_CLE,OffenseTeam_DAL,OffenseTeam_DEN,...,PassType_SHORT MIDDLE,PassType_SHORT RIGHT,PassType_[58-H.LANDRY III],RushDirection_CENTER,RushDirection_LEFT END,RushDirection_LEFT GUARD,RushDirection_LEFT TACKLE,RushDirection_RIGHT END,RushDirection_RIGHT GUARD,RushDirection_RIGHT TACKLE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [53]:
#still keep offense and defense team for further processing
final_data_dropped = final_data.drop(axis=1, columns=list(set(cat_columns)-set(["OffenseTeam", "DefenseTeam", "Formation"])))
final_data_dropped.reset_index(inplace=True)
print(final_data_dropped.shape)
print(final_data_dropped.head())

(114827, 18)
   index      GameId    GameDate  Quarter  Minute  Second OffenseTeam  \
0      0  2021092612  2021-09-26        1       9      10         MIN   
1      1  2021092612  2021-09-26        1       8      32         MIN   
2      2  2021092612  2021-09-26        1       7      52         MIN   
3      3  2021092612  2021-09-26        1       7      13         MIN   
4      4  2021101008  2021-10-10        1       9      50         WAS   

  DefenseTeam  Down  ToGo  YardLine  SeriesFirstDown  \
0         SEA     1    10        57                1   
1         SEA     1    10        74                0   
2         SEA     2     2        82                1   
3         SEA     1     8        92                0   
4          NO     2     2        51                1   

                                         Description  SeasonYear  Yards  \
0  (9:10) 8-K.COUSINS PASS SHORT RIGHT TO 83-T.CO...        2021     17   
1  (8:32) 8-K.COUSINS PASS SHORT LEFT TO 18-J.JEF...        2

In [54]:
#confirm no shared columns between one hot features and remaining not transformed columns
set(final_data_dropped.columns).intersection(one_hot_final.columns)

set()

### Combined non-categorical and categorical features to the final dataset

In [55]:
combined_final_data = pd.concat([final_data_dropped, one_hot_final], axis=1)
combined_final_data = combined_final_data.drop(axis=1, columns=['index'])
print(combined_final_data.shape)
print(combined_final_data.head())

(114827, 112)
       GameId    GameDate  Quarter  Minute  Second OffenseTeam DefenseTeam  \
0  2021092612  2021-09-26        1       9      10         MIN         SEA   
1  2021092612  2021-09-26        1       8      32         MIN         SEA   
2  2021092612  2021-09-26        1       7      52         MIN         SEA   
3  2021092612  2021-09-26        1       7      13         MIN         SEA   
4  2021101008  2021-10-10        1       9      50         WAS          NO   

   Down  ToGo  YardLine  ...  PassType_SHORT MIDDLE PassType_SHORT RIGHT  \
0     1    10        57  ...                    0.0                  1.0   
1     1    10        74  ...                    0.0                  0.0   
2     2     2        82  ...                    0.0                  0.0   
3     1     8        92  ...                    0.0                  0.0   
4     2     2        51  ...                    0.0                  0.0   

   PassType_[58-H.LANDRY III]  RushDirection_CENTER RushDire

In [56]:
#with open("20220320_play_by_play_finals.pkl", "wb") as f:
    pickle.dump(combined_final_data, f)
print("Finished!")

Finished!
